In [5]:
### colab 사용 시 실행
from google.colab import drive
drive.mount('/content/gdrive/')
import os
os.chdir('/content/gdrive/My Drive/Google Colaboratory/kakao_arena/kakao_arena_melon/src/ipynb')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [6]:
import sys
sys.path.append('../')
from basic_utils import *
from modeling_utils import *

from sklearn.metrics.pairwise import cosine_similarity
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.models import Model, load_model
from keras.initializers import glorot_normal, Zeros, Ones
import keras.backend as K
from keras.optimizers import RMSprop
import tensorflow as tf

In [ ]:
data_path = './../../../1_melon_playlist/data/'

train = pd.read_json(data_path+'train.json')
val = pd.read_json(data_path+'val.json')
# test = pd.read_json(data_path+'test.json')
song_meta = load_json(data_path+'song_meta.json')
# genre_gn_all = pd.read_json(data_path+'genre_gn_all.json', typ = 'series')

### make_file.ipynb 내에서 만들어진 tag_song_dict / song_tag_dict 불러오기
tag_song_dict = load_json(data_path+'tag_song_dict_no0.json')
song_tag_dict = load_json(data_path+'song_tag_dict.json')

### 생성해둔 song / plylst id에 대한 pivot table 불러오기
train_pv = pd.read_parquet(data_path+'train_DF.parquet')     ## colab에서 load조차 불가능..

In [ ]:
print(train_pv.shape)
display(train_pv.head(2))
display(train_pv.tail(2))

(34242, 18074)


id,6,25,39,50,55,85,106,107,112,127,...,153265,153275,153280,153288,153332,153347,153364,153388,153421,153428
song,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


id,6,25,39,50,55,85,106,107,112,127,...,153265,153275,153280,153288,153332,153347,153364,153388,153421,153428
song,,,,,,,,,,,,,,,,,,,,,
707913,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
707949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
## make a model
# this is the size of our encoded representations
# encoding_dim = 200  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# intput size
input_dim = train_pv.shape[1]
input_dim

18074

## denoising autoencoder

In [ ]:
a = 1000
b = 500

input_img = Input(shape=(input_dim,)) ## input layer
encoded = Dense(a, activation='relu')(input_img) ## encoding dim으로 압축
encoded = Dense(b, activation='relu')(encoded) ## encoding dim으로 압축

decoded = Dense(b, activation='sigmoid')(encoded) ## input dim으로 복귀
decoded = Dense(input_dim, activation='sigmoid')(encoded) ## input dim으로 복귀

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

# create a placeholder for an encoded (32-dimensional) input01
# encoded_input = Input(shape=(encoding_dim,))
encoded_input = Input(shape=(b,))

# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

# compile
autoencoder.compile(optimizer='adam', loss='mse') # (optimizer='adadelta', loss='binary_crossentropy')

## train
tmp = train_pv.values
history = autoencoder.fit(tmp, tmp,
                epochs=5,
                batch_size=256,
                shuffle=True)

Epoch 1/5
34242/34242 [==============================] - 167s 5ms/step - loss: 0.0283
Epoch 2/5
34242/34242 [==============================] - 161s 5ms/step - loss: 0.0013
Epoch 3/5
34242/34242 [==============================] - 163s 5ms/step - loss: 0.0013
Epoch 4/5
34242/34242 [==============================] - 171s 5ms/step - loss: 0.0013
Epoch 5/5
34242/34242 [==============================] - 162s 5ms/step - loss: 0.0013


In [ ]:
encoded_imgs = encoder.predict(tmp)
# decoded_imgs = decoder.predict(encoded_imgs)

del input_img, autoencoder, encoder, encoded_input, decoder_layer
gc.collect()

292

In [ ]:
del tmp
gc.collect()

20

In [ ]:
train_data = pd.DataFrame(data=encoded_imgs)

del encoded_imgs
gc.collect()

20

In [ ]:
cosine_array = cosine_similarity(train_data, train_data)
print('cosine_array complete')
# item_based_array= pd.DataFrame(data = cosine_array, index = train_pv.index, columns = train_pv.index)

index2id = {i:u for i, u in enumerate((train_pv.index))}

cf_dict = {}
for i in tqdm(range(len(cosine_array))):
    song_id= list(map(index2id.get, list(np.argsort(-cosine_array[i])[1:201])))
    value= list(np.sort(cosine_array[0])[::-1][1:201])
    cf_dict[index2id[i]] = list(zip(song_id,value))

del cosine_array, train_data
gc.collect()

  0%|          | 15/34242 [00:00<04:01, 141.87it/s]

cosine_array complete


100%|██████████| 34242/34242 [03:30<00:00, 162.85it/s]


0

## v3 Predict
- 기존에 존재하던 tag에서 가장 빈번하게 나타났던 음악을 넣어주기 

In [ ]:
no_tag, no_song, yes_index, no_both = check_target_type(val)

노래가 하나도 없음 :  2630
노래, 태그 둘 다 존재함 :  8975
태그가 없음 :  9661
둘다 없음 :  1749


In [ ]:
train['year'] = train['updt_date'].apply(lambda x: int(x[2:4]))
val['year'] = val['updt_date'].apply(lambda x: int(x[2:4]))

year_list = list(train['year'].unique())
year_list= sorted(year_list)

val3 = val[val.index.isin(yes_index)]

small_years = [17,18,19,20]

val_tmp = val3[val3['year'].isin(small_years)]
print("val year shape", val_tmp.shape)
val_tmp.reset_index(inplace = True, drop = True)

val year shape (6401, 7)


In [ ]:
popular_year = []
for i in year_list:
    tmp = train[train['year'] == i]
    _, year_song = most_popular(tmp, "songs", 200)
    _, year_tag = most_popular(tmp, "tags", 100)
    popular_year.append({
        "year" : i,
        "songs" : year_song,
        "tags" : year_tag,})

In [ ]:
val3_predict = []
for i in val_tmp.index:
    year = val_tmp.loc[i]['year']
    cur_song = add_song_by_autoencoder(val_tmp.loc[i], cf_dict, popular_year)
    # cur_song = add_song(val_tmp, i, cf_dict)
    cur_tag = add_var(val_tmp.loc[i], 'tags', 'songs', song_tag_dict, popular_year, year-4)
    # cur_tag = add_tag(val_tmp, i, song_tag_dict)
    val3_predict.append({
        "id" : val_tmp.loc[i]['id'],
        "songs": remove_seen(val_tmp.loc[i]['songs'], cur_song)[:100],
        "tags": remove_seen(val_tmp.loc[i]['tags'], cur_tag)[:10],
     })   

In [ ]:
write_json(val3_predict, 'val3_2019_predict.json')

---

In [ ]:
def f7(seq):
    '''
    list에 있는 중복 데이터는 삭제하고, 순서는 유지하는 함수
    set만 사용하면 순서가 뒤엉키게 됨
    '''
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

def add_song(val_tmp, i, cf_dict):
    year = val_tmp.loc[i]['year']
    songs = val_tmp.loc[i]['songs']
    songs = [x for x in songs if x in cf_dict]
    if len(songs):
        tmp_list = []
        for song in songs:
            tmp_list.extend(cf_dict[song])
        tmp_list.sort(key=lambda x : x[1], reverse=True)
        tmp_list = [i[0] for i in tmp_list if i[1] > 0]
        tmp_list = f7(tmp_list)
        cur_song = tmp_list[:200]
    else:
        cur_song = popular_year[year-4]['songs']
    cur_song = remove_seen(var_tmp.loc[i]['songs'], cur_song)[:100]
    return cur_song

def add_tag(val_tmp, i, song_tag_dict):
    tag_counter = Counter()
    for song in val_tmp.loc[i]['songs']:
        if str(song) in song_tag_dict:
            for tag in song_tag_dict[str(song)]:
                tag_counter.update({tag : 1})
    tag_counter = sorted(tag_counter.items(), key=lambda x : x[1], reverse=True)[:100]
    cur_tag = []
    for k in tag_counter:
        cur_tag.append(k[0])
    if len(cur_tag) == 0:
        cur_tag = popular_year[year-4]['tags'][:100]
    elif len(cur_tag) < 100:
        update_tag = remove_seen(cur_tag, popular_year[year-4]['tags'])
        cur_tag.extend(update_tag)
        cur_tag = cur_tag[:100]
    return cur_tag


In [ ]:
val3_predict = []
for i in val_tmp.index:
    year = val_tmp.loc[i]['year']
    
    ################ song 채우기 ###############################
    songs = val_tmp.loc[i:i, 'songs'].values[0]
    songs = [x for x in songs if x in cf_dict]

    if len(songs):
        tmp_list = []
        for song in songs:
            tmp_list.extend(cf_dict[song]) 
        tmp_list.sort(key = lambda x: x[1], reverse = True)    
        tmp_list = [i[0] for i in tmp_list if i[1]>0]
        tmp_list = f7(tmp_list)

        cur_song = tmp_list[:200]
    else:
        cur_song = popular_year[year-4]['songs']

    cur_song = remove_seen(list(val_tmp.loc[i:i, 'songs'])[0], cur_song)[:100]

    #중복 케이스 때문에 부족하면 popular song으로 다시 채워줌
    if len(cur_song) != 100:
        update_song = remove_seen(cur_song, popular_year[year-4]['songs'])
        cur_song.extend(update_song)
        cur_song = cur_song[:200]
    
    
    ############### tag 채우기 ##################################
    tag_counter = Counter()
    for song in list(val_tmp.loc[i:i, 'songs'].values[0]):
        if str(song) in song_tag_dict:
            for tag in song_tag_dict[str(song)]:
                tag_counter.update({tag : 1})
    tag_counter = sorted(tag_counter.items(), key= lambda x:x[1], reverse = True)        
    tag_counter = tag_counter[:100] ## 넉넉히 100개 잡는다. 
    
    cur_tag = []
    for k in tag_counter:
        cur_tag.append(k[0])     
        
    if (len(cur_tag) < 100) & (len(cur_tag) > 0 ):
        ## 부족하면 채워준다.
        update_tag = remove_seen(cur_tag, popular_year[year-4]['tags'])
        cur_tag.extend(update_tag)
        cur_tag = cur_tag[:100]
    if len(cur_tag) == 0:
        cur_tag = popular_year[year-4]['tags'][:100]
        
    #############################################################    

    val3_predict.append({
        "id" : list(val_tmp.loc[i:i, 'id'])[0],
        "songs": remove_seen(list(val_tmp.loc[i:i, 'songs'])[0], cur_song)[:100],
        "tags": remove_seen(list(val_tmp.loc[i:i, 'tags'])[0], cur_tag)[:10],
     })    
